# <u>Sea Level Rise Threat Index Tool </u>

#### Meg Manning -- *Duke University, Fall 2024*

► **Description:** this script can be used to identify an area in the coastal plain of North Carolina and assess it's risk to sea level rise. The overall threat index is calculated through combining the ranked exposure of inundation under 1ft, 2ft, and 3ft of sea level rise, flood prone areas, storm surge under category 1-5 hurricanes, soil erodibility, soil drainage, and areas of low slope for a given area of interest. Methods for forecasting sea level rise inundation were taken from the National Oceanic and Atmospheric detailed methology for sea level rise inundation (NOAA, 2017) and the overall threat index was modelled after the National Fish and Wildlife Service's Coastal Resilience and Siting Tool (CREST) (Dobson et al., 2019).

### 1. Digital Elevation Model from LiDAR 

*Download LiDAR point data and process into a 1m DEM.*

#### Since there is no API for a direct download of LiDAR point data, in order to process the rest of this script, data must be downloaded following these steps *prior* to running the rest of this script. 

<u> Step 1: Download LiDAR Data </u> 

Click the link below to open the Digital Coast Topobathy LiDAR Web Interface: https://coast.noaa.gov/dataviewer/#/lidar/search/ 

<u>Step 1.1:</u> Zoom in to a scale of at least 1000ft, if not 500ft, and draw a box around the area of interst

<u>Step 1.2:</u> In the panel on the right, select the most recent *point* data that corresponds to the area of interest

<u>Step 1.3:</u> Click "Add to Cart" 

<u>Step 1.4:</u> In your cart, set the following parameters for your point data:
* Projection = UTM
* Zone = Zone 18 Range 078W-072W
* Horizontal Datum = NAD83
* Horizontal Units = Meters
* Vertical Datum = NADV88
* Vertical Units = Meters
* Output Product = Point
* Output Format = Points - LAS
* Data Classes = Ground 

<u>Step 1.5:</u> Click next, add email, review and submit

*It may take a few minutes for the data to process but you will recieve an email with the link to download the data. Once it downloads, extract the data to your machine and note the file path.*

<u> Step 2: Process into a 1m DEM</u> 

In [24]:
# Load Necessary Packages
import arcpy
import os

# Set the workspace 
#arcpy.env.workspace = "C:/path_to_lidar_data"
arcpy.env.workspace = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351"
arcpy.env.overwriteOutput = True

In [25]:
# Define LAS data and LAS dataset paths 
#**Can I make these user inputs?**
#las_folder = "C:/path_to_lidar_data"
#las_dataset = "C:/path_to_lidar_data/las_dataset.lasd"
las_folder = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351"
las_dataset = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351/nc2019_dunex_Job1099351.lasd"

# Create a LAS Dataset using Create LAS Dataset Tool 
arcpy.management.CreateLasDataset(las_folder, las_dataset)

<Result 'C:\\Users\\mgm79\\Downloads\\nc2019_dunex_Job1099351\\nc2019_dunex_Job1099351.lasd'>

In [26]:
# Define output raster (DEM) path 
ground_DEM = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351/DEM_1m.tif"

# Create 1m DEM using LAS Dataset to Raster tool 
arcpy.conversion.LasDatasetToRaster(las_dataset, ground_DEM, 
                                    'ELEVATION', 
                                    'BINNING AVERAGE LINEAR', 
                                    'FLOAT', 
                                    'CELLSIZE', 1, 1)


<Result 'C:\\Users\\mgm79\\Downloads\\nc2019_dunex_Job1099351\\DEM_1m.tif'>

### 2. Tidal Variability Surface Using Vertical Datum Tool

*Use DEM created above to create a tidal surface raster that shows the tidal fluctuations at the area of interest. Once this is created then sea level rise inundation can be calculated.*

In [ ]:
# Create Points from DEM to input into VDatum tool to get conversion from MHHW into NAVD88

# Raster to Point - set local variables 
in_DEM = ground_DEM
out_point_features = 'DEM_points.shp'
field = "VALUE"

# Run the tool
# arcpy.conversion.RasterToPoint(in_DEM, out_point_features, field)

#IS THERE AN EASIER WAY TO DO THIS?

### 3. Calculate Inundation Extent under Desired SLR Amount

*Use both the DEM in step 1 and the tidal variabliity surface in step 2 to model the desired amount of sea level rise at 1 meter resolution on top of the area of interest. For detailed methods see Deatiled Method for Mappnig Sea Level Rise Inundation (NOAA, 2017).*

<u> Step 3.1: Add desired SLR amount to tidal raster </u>

In [49]:
import arcpy
from arcpy.ia import *
from arcpy.sa import *
from arcpy.sa import Con

# Allow outputs to be overwritten
arcpy.env.overwriteOutput = True

# Check out any necessary licenses.
arcpy.CheckOutExtension("3D")
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")

# Set local variables
ground_DEM = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351/DEM_1m.tif"
#tidal_raster = arcpy.Raster("tidal_variability.tif")
SLR_1ft = 0.3048 
SLR_2ft = 0.6096
SLR_3ft = 0.9144 
out_surface = '../Scratch/surface.tif'

# Execute Plus tool 
surface_raster = arcpy.ddd.Plus(in_raster_or_constant1=ground_DEM, 
               in_raster_or_constant2=SLR_1ft, 
               out_raster=out_surface)

<u> Step 3.2: To evaluate connectivity, create single value DEM to show inundation extent </u>

In [ ]:
# HOW DO I LOAD THESE RASTERS SO THAT THEY'RE RASTERS...
print(type(ground_DEM))
print(type(surface_raster))

<class 'str'>
<class 'arcpy.arcobjects.arcobjects.Result'>


In [ ]:
# Raster Calculator - 
single = Con(ground_DEM <= surface_raster, 1)
single.save("../Scratch/single.tif")

RuntimeError: The parameter is incorrect. 

<u> Step 3.3: Evaluate connectivity of extent raster using region group  </u>

In [8]:
# Region Group
clumped = "../Scratch/clumped.tif"
Region_Group = clumped
clumped = arcpy.sa.RegionGroup(single, "EIGHT", "WITHIN", "NO_LINK", None)
clumped.save(Region_Group)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000865: Input raster: ../Scratch/single.tif does not exist.
Failed to execute (RegionGroup).


<u> Step 3.4: Extract connected inundation surface to be used as a mask for the original depth grid  </u>

In [ ]:
# Extract by Attributes
lowlying = "../Scratch/lowlying.tif"
Extract_by_Attributes = lowlying
lowlying = arcpy.sa.ExtractByAttributes(clumped, "Count < 1000000")
lowlying.save(Extract_by_Attributes)

<u> Step 3.5: Subtract DEM values from water surface to derive initial inundatino depth grid </u>

In [ ]:
# Raster Calculator (2)
depth = "../Scratch/depth.tif"
Raster_Calculator_2 = depth
depth =  Con(ground_DEM <= surface_raster, surface_raster - ground_DEM) 
depth.save(Raster_Calculator_2)

<u> Step 3.6: Derive low-lying areas  </u>

In [ ]:
# Extract by Attributes (2) --> THIS IS GOING TO DEPEND ON THE AREA??
connect = "../Scratch/connect.tif"
Extract_by_Attributes_2 = connect
connect = arcpy.sa.ExtractByAttributes(clumped, "Count > 1000000")
connect.save(Extract_by_Attributes_2)

<u> Step 3.7: Create depth grid for connected areas </u>

In [ ]:
# Extract by Mask
con_depth = "../Scratch/con_depth.tif"
Extract_by_Mask = con_depth
con_depth = arcpy.sa.ExtractByMask(depth, connect, "INSIDE", "221627.8233 3780773.4727 224415.3247 3783394.4331 PROJCS[\"NAD_1983_2011_UTM_Zone_18N\",GEOGCS[\"GCS_NAD_1983_2011\",DATUM[\"D_NAD_1983_2011\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",500000.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-75.0],PARAMETER[\"Scale_Factor\",0.9996],PARAMETER[\"Latitude_Of_Origin\",0.0],UNIT[\"Meter\",1.0]],VERTCS[\"NAVD_1988\",VDATUM[\"North_American_Vertical_Datum_1988\"],PARAMETER[\"Vertical_Shift\",0.0],PARAMETER[\"Direction\",1.0],UNIT[\"Meter\",1.0]]")
con_depth.save(Extract_by_Mask)

In [ ]:
# Change the working environment to where rasters are saved
# Add user input for this? arcpy.env.workspace = "C:/Users/mgm79/Downloads/nc2019_dunex_Job1099351"
from arcpy import sa

# Set local variables
#tidal_DEM 
#DEM_output 
SLR_1ft = 0.3048 
SLR_2ft = 0.6096
SLR_3ft = 0.9144 

#output_plus = "surface_SLR_1ft.tif" #create this as a function of what the user inputs for SLR amount
# Add desired amount of SLR to tidal variability raster  --> CAN THIS ALSO BE A USER INPUT?
## Remember this will be in *meters* not feet

# Plus tool 
outPlus = sa.Plus(ground_DEM, SLR_1ft)

# Save the output
outPlus.save('../Scratch/surface.tif')

# View the new surface 
#outPlus

### 4. Scrape & Process Threat Index Layers 

*Prepare each dataset following the methods detailed in Dobson et al. (2019) to create an index of overall threat from sea level rise.*

### Flood Prone Areas 

Extract the National Flood Hazard Layer (NFHL) from FEMA Flood Prone Areas database (FEMA, 2022).

In [1]:
# Import necessary packages 
import pandas as pd

from arcgis import GIS
from arcgis.geocoding import geocode
from arcgis.mapping import WebMap, MapServiceLayer, MapFeatureLayer
from arcgis.features import FeatureLayer, GeoAccessor
from arcgis.geometry import filters

gis = GIS('home')

In [2]:
#Get the web mapping application item FEMA provides
fema_web_map_app = gis.content.get('8b0adb51996444d4879338b5529aa9cd')
fema_web_map_app.type

'Web Mapping Application'

In [3]:
#Get the data associated with the item
fema_web_map_app_data = fema_web_map_app.get_data()

#From the dictionary, extract the map's item ID
fema_web_map_app_ID = fema_web_map_app_data['map']['itemId']

#Find the item associated with that item 
fema_web_map_item = gis.content.get(fema_web_map_app_ID)
fema_web_map_item.type

'Web Map'

In [4]:
#Create a web map from the web map item
fema_web_map = WebMap(fema_web_map_item)

#Extract its layers
fema_web_map_layers = fema_web_map.layers

#Show the layer titles
[layer['title'] for layer in fema_web_map_layers]

['Effective FIRM Panels',
 'NFHL',
 'Coastal Barrier Resources System Area (US FWS)']

In [5]:
#Get the URL for the MapServiceLayer
the_layer = fema_web_map.layers[1]
the_layer_url = the_layer.url
print(the_layer_url)

# Grab the Flood Hazard Zones
the_service_url = the_layer_url + '/20'

https://hazards.fema.gov/arcgis/rest/services/FIRMette/NFHLREST_FIRMette/MapServer


In [6]:
#Create a MapServiceLayer object from the url
fema_ms_layer = MapServiceLayer(fema_web_map_layers[1]['url'])
fema_layers = fema_ms_layer.properties['layers']
len(fema_layers)

32

In [7]:
#Fetch the layer
nhfl_layer = FeatureLayer(the_service_url)
nhfl_layer

<FeatureLayer url:"https://hazards.fema.gov/arcgis/rest/services/FIRMette/NFHLREST_FIRMette/MapServer/20">

In [ ]:
# CAN I MAKE THIS A USER INPUT FOR THE COUNTY THE AREA OF INTEREST IS LOCATED IN?

#Create a spatial filter for Hyde County
county_lyr = gis.content.get('14c5450526a8430298b2fa74da12c2f4').layers[0]
county_sdf = county_lyr.query("STATE_NAME = 'North Carolina' AND NAME = 'Hyde County'").sdf
county_geom = county_sdf.iloc[0,-1]
county_filter = filters.intersects(county_geom)

In [12]:
#Spatially query the nhfl feature layers
nhfl_fset = nhfl_layer.query(geometry_filter=county_filter)
nhfl_fset

TypeError: 'FeatureSet' object is not subscriptable

In [ ]:
#Convert to sdf
nhfl_sdf = nhfl_fset.sdf
nhfl_sdf.spatial.to_featureclass('../Scratch/Hyde_NHFL.shp')

### Storm Surge

Extract data for storm surge extents under a category 1-5 hurricane for the area of interest. This data is from the National Hurricane Center (Zachry, B. C. et al. 2015).

In [ ]:
# Get the layers for storm surge under cat. 1-5 storms 
storm_surge_cat1_URL = gis.content.get()

In [18]:
# Read the data in as a feature layer 
storm_surge_cat1 = FeatureLayer(storm_surge_cat1_URL)
type(storm_surge_cat1)

arcgis.features.layer.FeatureLayer

In [21]:
# Conert to spatial dataframe
storm_surge_1_sdf  = GeoAccessor.from_layer(storm_surge_cat1)


Exception: Malformed response from server, could not load the dataset: Expecting value: line 1 column 1 (char 0)

In [22]:
try:
    storm_surge_1_sdf = GeoAccessor.from_layer(storm_surge_cat1)
except Exception as e:
    print(f"Error: {e}")

Error: Malformed response from server, could not load the dataset: Expecting value: line 1 column 1 (char 0)


### Areas of Low Slope

Calculate the slope of the Digital Elevation Model and reclassify the raster to create a rank for areas of low slope. 

In [ ]:
# Set local variables 
in_raster = ground_DEM
output_measurement = "PERCENT_RISE"

'CheckedOut'

In [ ]:
# Execute Slope
outSlope = arcpy.sa.Slope(in_raster, output_measurement)
outSlope.save("../Scratch/slope.tif") #will probably have to change this with input?

In [43]:
# Set local variables 
from arcpy.sa import RemapRange
remap = RemapRange([[0,0.25, 5],
                   [0.25, 0.50, 4],
                   [0.50,0.75, 3], 
                   [0.75,1, 2],
                   [1,2, 1],
                   [2,100, 0]])

# Reclassify Slope Raster 
reclassified_slope = arcpy.sa.Reclassify(outSlope, "VALUE", remap, "NODATA")

#Save the new file 
reclassified_slope.save("../Scratch/slope_ranked.tif")

#### References 

#### Dobson, G., Johnson, I., Rhodes, K., Hutchins, M., & Chesnutt, M. (2019). Regional coastal resilience assessment: Methodology. National Fish and Wildlife Foundation. Retrieved November 21, 2024, from https://www.nfwf.org/sites/default/files/coastalresilience/Documents/regional-coastal-resilience-assessment.pdf

#### Federal Emergency Management Agency (FEMA). (2022). National Flood Hazard Layer (NFHL). FEMA. Retrieved November 24, 2024, from https://hazards.fema.gov/femaportal/wps/portal/NFHLWMS

#### National Oceanic and Atmospheric Administration. (2017, January). Sea level rise inundation methods: Technical documentation. NOAA Office for Coastal Management. Retrieved November 21, 2024, from https://coast.noaa.gov/data/digitalcoast/pdf/slr-inundation-methods.

#### Zachry, B. C., W. J. Booth, J. R. Rhome, and T. M. Sharon, 2015: A National View of Storm Surge Risk and Inundation. Weather, Climate, and Society, 7(2), 109–117. DOI: http://dx.doi.org/10.1175/WCAS–D–14–00049.1